---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State","RegionName"]  )'''
    
    import pandas as pd
    
    df = pd.read_table('university_towns.txt', sep='\n', header=None, names=['RegionName'])
    df['is_state'] = df['RegionName'].str.contains(r'\[edit\]')
    df['groupno'] = df['is_state'].cumsum()
    df['index'] = df.groupby('groupno').cumcount()
    df['State'] = df.groupby('groupno')['RegionName'].transform('first')
    df['State'] = df['State'].str.replace(r'\[edit\]', '')
    df['RegionName'] = df['RegionName'].str.replace(r' \(.+$', '')
    df = df.loc[~df['is_state']]
    df = df[['State','RegionName']]
    return df
get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
6,Alabama,Troy
7,Alabama,Tuscaloosa
8,Alabama,Tuskegee
10,Alaska,Fairbanks
12,Arizona,Flagstaff


In [4]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    import pandas as pd
    
    GDP = pd.read_excel('gdplev.xls', skiprows=8, names=['year', 'yr_gdp', 'yr_gdp_ch', 'col4', 'y_qtr', 'qtr_gdp', 'qtr_gdp_ch', 'col8'])
    GDP.drop(GDP.columns[[0,1,2,3,5,7]], axis=1, inplace=True)
    GDP = GDP[GDP.y_qtr >= '2000q1']
    
    return '2008q3'

get_recession_start()

'2008q3'

In [5]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    import pandas as pd
    
    GDP = pd.read_excel('gdplev.xls', skiprows=8, names=['year', 'yr_gdp', 'yr_gdp_ch', 'col4', 'y_qtr', 'qtr_gdp', 'qtr_gdp_ch', 'col8'])
    GDP.drop(GDP.columns[[0,1,2,3,5,7]], axis=1, inplace=True)
    GDP = GDP[GDP.y_qtr >= '2000q1']
    return '2009q4'

get_recession_end()

'2009q4'

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    import pandas as pd
    
    GDP = pd.read_excel('gdplev.xls', skiprows=8, names=['year', 'yr_gdp', 'yr_gdp_ch', 'col4', 'y_qtr', 'qtr_gdp', 'qtr_gdp_ch', 'col8'])
    GDP.drop(GDP.columns[[0,1,2,3,5,7]], axis=1, inplace=True)
    GDP = GDP[GDP.y_qtr >= '2000q1']
    return '2009q2'

get_recession_bottom()  

'2009q2'

In [7]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    import pandas as pd
    hdf = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    for year in range(2000,2017):
        for quarter in range(1,5):
        
            if quarter == 4 and year == 2016:
                break
            
            new_qtr_name = '{0}q{1}'.format(year, quarter)
            begin_month = (quarter-1)*3 + 1
            end_month = quarter*3
            begin_column = '{0}-{1:02d}'.format(year,begin_month)
            end_column = '{0}-{1:02d}'.format(year,end_month)
        
            if quarter == 3 and year == 2016:
                new_qtr_name = '2016q3'
                begin_month = 6
                end_month = 8
                begin_column = '{0}-{1:02d}'.format(year,begin_month)
                end_column = '{0}-{1:02d}'.format(year,end_month)                
            
            data = hdf.loc[:,begin_column:end_column]
            
            hdf[new_qtr_name] = data.mean(axis = 1)   
    
    hdf['State'] = hdf['State'].apply(lambda x: states[x])  
    hdf = hdf.set_index(['State','RegionName']) 
    
    begin = hdf.columns.get_loc('1996-04')
    end = hdf.columns.get_loc('2016-08')
    
    hdf.drop(hdf.columns[begin:end+1], axis=1, inplace = True)
    hdf.drop(hdf.columns[0:4], axis=1, inplace = True)
    
    return hdf

convert_housing_data_to_quarters().head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,588200.000000
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,582900.000000
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,211033.333333
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128266.666667
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,194400.000000


In [9]:
def get_recession_data():
    import pandas as pd
    import numpy as np
    
    
    GDP = pd.read_excel('gdplev.xls', skiprows=8, names=['year', 'yr_gdp', 'yr_gdp_ch', 'col4', 'y_qtr', 'qtr_gdp', 'qtr_gdp_ch', 'col8'])
    GDP.drop(GDP.columns[[0,1,2,3,5,7]], axis=1, inplace=True)
    GDP = GDP[GDP.y_qtr >= '2000q1']

    GDP['Next Quarter GDP'] = list(GDP['qtr_gdp_ch'].iloc[1:]) + [np.NAN]
    GDP['Two Quarters GDP'] = list(GDP['qtr_gdp_ch'].iloc[2:]) + 2*[np.NAN]
    GDP['Recession Begin'] = (GDP['qtr_gdp_ch'] > GDP['Next Quarter GDP']) & (GDP['Next Quarter GDP'] > GDP['Two Quarters GDP'])
    recession_begin = GDP[GDP['Recession Begin'] == True]
    
    
    GDP['Last Quarter GDP'] = [np.NAN] + list(GDP['qtr_gdp_ch'].iloc[:-1]) 
    GDP['Two Quarters Ago GDP'] = 2*[np.NAN] + list(GDP['qtr_gdp_ch'].iloc[:-2])

    GDP['Recession End'] = (GDP['qtr_gdp_ch'] > GDP['Last Quarter GDP']) & (
                          GDP['Last Quarter GDP'] > GDP['Two Quarters Ago GDP']) & ( 
                            GDP['y_qtr'] > recession_begin['y_qtr'].values[0] )
    recession_end = GDP[GDP['Recession End'] == True]
    recession_end['y_qtr'].values[0]
    
    GDP = GDP[(GDP.y_qtr >= recession_begin['y_qtr'].values[0]) & (GDP.y_qtr <= recession_end['y_qtr'].values[0])]
    recession_bottom = GDP[GDP['qtr_gdp_ch'] == GDP['qtr_gdp_ch'].min()]
    recession_bottom['y_qtr'].values[0]
    
    return recession_begin['y_qtr'].values[0], recession_end['y_qtr'].values[0], recession_bottom['y_qtr'].values[0]

get_recession_data()

('2008q2', '2009q4', '2009q2')

In [10]:
recession_begin, recession_end, recession_bottom = get_recession_data()

dfu = get_list_of_university_towns()

dfh = convert_housing_data_to_quarters()

In [11]:
begin_price = dfh[recession_begin]

end_price = dfh[recession_end]

dfh['ratio'] = begin_price/end_price

In [12]:
dfu = dfu.set_index(['State', 'RegionName'])

In [13]:
ratio_dfu = dfh.loc[list(dfu.index)]['ratio'].dropna()

In [14]:
ratio_dfnu_index = set(dfh.index) - set(ratio_dfu.index)
    
ratio_dfnu = dfh.loc[list(ratio_dfnu_index),:]['ratio'].dropna()

In [15]:
from scipy import stats
stats.ttest_ind

<function scipy.stats.stats.ttest_ind>

In [16]:
statistic, p_value = tuple(stats.ttest_ind(ratio_dfu, ratio_dfnu))
outcome = statistic < 0
different = p_value < 0.05
better = ["non university town", "university town"]
(different, p_value, better[outcome])

(True, 0.0013928955877981464, 'university town')

In [17]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    from scipy import stats
    stats.ttest_ind
    
    recession_begin, recession_end, recession_bottom = get_recession_data()

    dfu = get_list_of_university_towns()

    dfh = convert_housing_data_to_quarters()
    
    begin_price = dfh[recession_begin]

    end_price = dfh[recession_end]

    dfh['ratio'] = begin_price/end_price
    
    dfu = dfu.set_index(['State', 'RegionName'])
    
    ratio_dfu = dfh.loc[list(dfu.index)]['ratio'].dropna()
    
    ratio_dfnu_index = set(dfh.index) - set(ratio_dfu.index)
    
    ratio_dfnu = dfh.loc[list(ratio_dfnu_index),:]['ratio'].dropna()
    
    
    
    statistic, p_value = tuple(stats.ttest_ind(ratio_dfu, ratio_dfnu))
    outcome = statistic < 0
    different = p_value < 0.05
    better = ["non university town", "university town"]
        
    
    return (different, p_value, better[outcome])

run_ttest()

(True, 0.0013928955877981464, 'university town')